In [3]:
import sys

sys.path.append("/gstore/project/paneldesign/pkg")

sys.path.append("/gstore/home/taol9/gitsss/yo/tao")
# import primer3
from Bio import SeqIO
from Bio.Seq import Seq
from Bio import Entrez
import csv
from collections import defaultdict,OrderedDict
from collections import Counter
import matplotlib.pyplot as plt

import pandas as pd
import os
import pickle

from mybasic import rc, RandomDNA


# TSO(25nt)-flankingsequence(25nt)-Index1-UMI-repeat1-spacer(4nt)-repeat2-spacer-repeat3-spacer-repeat4-...repeatX-polyA;

## define adaptor


In [287]:
BC001 = "ACTTTAGG"
BC002 = "AACGGGAA"
BC003 = "AGTAGGCT"
BC004 = "ATGTTGAC"
BC005 = "ACAGACCT"
BC006 = "ATCCCAAC"
BC007 = "AAGTAGAG"
BC008 = "AGCTGTGA"
BC009 = "ACAGTCTG"
BC010 = "AGTGAGTG"
BC011 = "AGAGGCAA"
BC012 = "ACTACTCA"
BC013 = "ATACGTCA"
BC014 = "ATCATGTG"
BC015 = "AACGCCGA"
BC016 = "ATTCGGTT"

Mly1_R_w='GCTTCCTGATGAGTCCGATG'
Mly1_F_w='GCAACGACTCCACACTCATA'

In [ ]:
# TSO(25nt)-flankingsequence(25nt)-Index1-UMI-repeat1-spacer(4nt)-repeat2-spacer-repeat3-spacer-repeat4-...repeatX-polyA;

In [ ]:

spacer=RandomDNA(4)
units=["C","TG","AG"]
str_len=[6,9,15,18]
UMI="N"*6
index=BC003
TSO='AAGCAGTGGTATCAACGCAGAGTACAT'
dialout_adaptor_F=Mly1_F_w='GCAACGACTCCACACTCATA'
dialout_adaptor_R=Mly1_R_w='GCTTCCTGATGAGTCCGATG'
l=TSO+dialout_adaptor_F+index+UMI

while len(l)<2000:
    l=l+random.choice(str_len)*random.choice(units)+spacer

    

    
    

In [288]:
# use B013
# use LNA base primer for snv_dialout when run with rna probes
# max lenght set to 120 for rna probes
BC013 = "ATACGTCA"
snv_dialout_adaptor=Mly1_F_w='GCAACGACTCCACACTCATA'

def flex_padlock(template,mutation,probeset_barcode_8nt="ATACGTCA",snv_dialout_adaptor='GCAACGACTCCACACTCATA'):
    target_LHS=get_target_LHS(template,mutation)

    target_RHS=get_target_RHS(template,mutation)
    left_arm0=snv_dialout_adaptor+"CCTTGGCACCCGAGAATTCCA"+target_LHS[0]
    left_arm1=snv_dialout_adaptor+"CCTTGGCACCCGAGAATTCCA"+target_LHS[1]

    right_arm="/5Phos/"+target_RHS+"ACGCGGTTAGCACGTANN"+probeset_barcode_8nt+"CGGTCCTAGCAA"
    #fulllib:
    #p7+i7+left_arm+right_arm+10xumi+10xcellbarcode+read1seqprimer+i5+p5
#     left_arm_snv=snv_dialout_adaptor+"CCTTGGCACCCGAGAATTCCA"+target_LHS
    padlock_ref=right_arm+snv_dialout_adaptor+"CCTTGGCACCCGAGAATTCCA"+target_LHS[0]
    padlock_alt=right_arm+snv_dialout_adaptor+"CCTTGGCACCCGAGAATTCCA"+target_LHS[1]
#     print (len(target_RHS+"ACGCGGTTAGCACGTANN"+probeset_barcode_8nt+"CGGTCCTAGCAA")+len(left_arm0), len(target_LHS[0]),len(target_RHS))

    return {'LHS_ref':left_arm0, 'target_LHS_ref':target_LHS[0], 'LHS_mut':left_arm1, 'target_LHS_mut':target_LHS[1],'target_RHS':target_RHS,"RHS":right_arm, "padlock_ref":padlock_ref, "padlock_alt":padlock_alt,}

In [219]:
# convert to padlock probe

In [245]:
KRAS_CDS='''ATGACTGAATATAAACTTGTGGTAGTTGGAGCTGGTGGCGTAGGCAAGAGTGCCTTGACGATACAGCTAATTCAGAATCATTTTGTGGACGAATATGATCCAACAATAGAGGATTCCTAC
AGGAAGCAAGTAGTAATTGATGGAGAAACCTGTCTCTTGGATATTCTCGACACAGCAGGT
CAAGAGGAGTACAGTGCAATGAGGGACCAGTACATGAGGACTGGGGAGGGCTTTCTTTGT
GTATTTGCCATAAATAATACTAAATCATTTGAAGATATTCACCATTATAGAGAACAAATT
AAAAGAGTTAAGGACTCTGAAGATGTACCTATGGTCCTAGTAGGAAATAAATGTGATTTG
CCTTCTAGAACAGTAGACACAAAACAGGCTCAGGACTTAGCAAGAAGTTATGGAATTCCT
TTTATTGAAACATCAGCAAAGACAAGACAGAGAGTGGAGGATGCTTTTTATACATTGGTG
AGAGAGATCCGACAATACAGATTGAAAAAAATCAGCAAAGAAGAAAAGACTCCTGGCTGT
GTGAAAATTAAAAAATGCATTATAATGTAA'''
KRAS_CDS=KRAS_CDS.replace('\n','')



In [260]:
KRAS_flex=flex_padlock(KRAS_CDS,'c.34G>T')
KRAS_flex

118 22 22


{'LHS_ref': 'GACTCCACACTCATACCTTGGCACCCGAGAATTCCAAGGCACTCTTGCCTACGCCACC',
 'target_LHS_ref': 'AGGCACTCTTGCCTACGCCACC',
 'LHS_mut': 'GACTCCACACTCATACCTTGGCACCCGAGAATTCCAGGCACTCTTGCCTACGCCACCA',
 'target_LHS_mut': 'GGCACTCTTGCCTACGCCACCA',
 'target_RHS': 'AGCTCCAACTACCACAAGTTTA',
 'RHS': '/5Phos/AGCTCCAACTACCACAAGTTTAACGCGGTTAGCACGTANNATACGTCACGGTCCTAGCAA',
 'padlock_ref': '/5Phos/AGCTCCAACTACCACAAGTTTAACGCGGTTAGCACGTANNATACGTCACGGTCCTAGCAAGACTCCACACTCATACCTTGGCACCCGAGAATTCCAAGGCACTCTTGCCTACGCCACC',
 'padlock_alt': '/5Phos/AGCTCCAACTACCACAAGTTTAACGCGGTTAGCACGTANNATACGTCACGGTCCTAGCAAGACTCCACACTCATACCTTGGCACCCGAGAATTCCAGGCACTCTTGCCTACGCCACCA'}

In [91]:
TP63_CDS='''ATGAATTTTGAAACTTCACGGTGTGCCACCCTACAGTACTGCCCTGACCCTTACATCCAGCGTTTCGTAG
AAACCCCAGCTCATTTCTCTTGGAAAGAAAGTTATTACCGATCCACCATGTCCCAGAGCACACAGACAAA
TGAATTCCTCAGTCCAGAGGTTTTCCAGCATATCTGGGATTTTCTGGAACAGCCTATATGTTCAGTTCAG
CCCATTGACTTGAACTTTGTGGATGAACCATCAGAAGATGGTGCGACAAACAAGATTGAGATTAGCATGG
ACTGTATCCGCATGCAGGACTCGGACCTGAGTGACCCCATGTGGCCACAGTACACGAACCTGGGGCTCCT
GAACAGCATGGACCAGCAGATTCAGAACGGCTCCTCGTCCACCAGTCCCTATAACACAGACCACGCGCAG
AACAGCGTCACGGCGCCCTCGCCCTACGCACAGCCCAGCTCCACCTTCGATGCTCTCTCTCCATCACCCG
CCATCCCCTCCAACACCGACTACCCAGGCCCGCACAGTTTCGACGTGTCCTTCCAGCAGTCGAGCACCGC
CAAGTCGGCCACCTGGACGTATTCCACTGAACTGAAGAAACTCTACTGCCAAATTGCAAAGACATGCCCC
ATCCAGATCAAGGTGATGACCCCACCTCCTCAGGGAGCTGTTATCCGCGCCATGCCTGTCTACAAAAAAG
CTGAGCACGTCACGGAGGTGGTGAAGCGGTGCCCCAACCATGAGCTGAGCCGTGAATTCAACGAGGGACA
GATTGCCCCTCCTAGTCATTTGATTCGAGTAGAGGGGAACAGCCATGCCCAGTATGTAGAAGATCCCATC
ACAGGAAGACAGAGTGTGCTGGTACCTTATGAGCCACCCCAGGTTGGCACTGAATTCACGACAGTCTTGT
ACAATTTCATGTGTAACAGCAGTTGTGTTGGAGGGATGAACCGCCGTCCAATTTTAATCATTGTTACTCT
GGAAACCAGAGATGGGCAAGTCCTGGGCCGACGCTGCTTTGAGGCCCGGATCTGTGCTTGCCCAGGAAGA
GACAGGAAGGCGGATGAAGATAGCATCAGAAAGCAGCAAGTTTCGGACAGTACAAAGAACGGTGATGGTA
CGAAGCGCCCGTTTCGTCAGAACACACATGGTATCCAGATGACATCCATCAAGAAACGAAGATCCCCAGA
TGATGAACTGTTATACTTACCAGTGAGGGGCCGTGAGACTTATGAAATGCTGTTGAAGATCAAAGAGTCC
CTGGAACTCATGCAGTACCTTCCTCAGCACACAATTGAAACGTACAGGCAACAGCAACAGCAGCAGCACC
AGCACTTACTTCAGAAACAGACCTCAATACAGTCTCCATCTTCATATGGTAACAGCTCCCCACCTCTGAA
CAAAATGAACAGCATGAACAAGCTGCCTTCTGTGAGCCAGCTTATCAACCCTCAGCAGCGCAACGCCCTC
ACTCCTACAACCATTCCTGATGGCATGGGAGCCAACATTCCCATGATGGGCACCCACATGCCAATGGCTG
GAGACATGAATGGACTCAGCCCCACCCAGGCACTCCCTCCCCCACTCTCCATGCCATCCACCTCCCACTG
CACACCCCCACCTCCGTATCCCACAGATTGCAGCATTGTCAGTTTCTTAGCGAGGTTGGGCTGTTCATCA
TGTCTGGACTATTTCACGACCCAGGGGCTGACCACCATCTATCAGATTGAGCATTACTCCATGGATGATC
TGGCAAGTCTGAAAATCCCTGAGCAATTTCGACATGCGATCTGGAAGGGCATCCTGGACCACCGGCAGCT
CCACGAATTCTCCTCCCCTTCTCATCTCCTGCGGACCCCAAGCAGTGCCTCTACAGTCAGTGTGGGCTCC
AGTGAGACCCGGGGTGAGCGTGTTATTGATGCTGTGCGATTCACCCTCCGCCAGACCATCTCTTTCCCAC
CCCGAGATGAGTGGAATGACTTCAACTTTGACATGGATGCTCGCCGCAATAAGCAACAGCGCATCAAAGA
GGAGGGGGAGTGA'''.replace('\n','')

In [250]:
TP63_flex=flex_padlock(TP63_CDS,'c.1090G>A')
TP63_flex

118 22 22


{'LHS_ref': 'GACTCCACACTCATACCTTGGCACCCGAGAATTCCACGTTCTTTGTACTGTCCGAAAC',
 'target_LHS_ref': 'CGTTCTTTGTACTGTCCGAAAC',
 'LHS_mut': 'GACTCCACACTCATACCTTGGCACCCGAGAATTCCAGTTCTTTGTACTGTCCGAAACT',
 'target_LHS_mut': 'GTTCTTTGTACTGTCCGAAACT',
 'target_RHS': 'TTGCTGCTTTCTGATGCTATCT',
 'RHS': '/5Phos/TTGCTGCTTTCTGATGCTATCTACGCGGTTAGCACGTANNATACGTCACGGTCCTAGCAA',
 'padlock_ref': '/5Phos/TTGCTGCTTTCTGATGCTATCTACGCGGTTAGCACGTANNATACGTCACGGTCCTAGCAAGACTCCACACTCATACCTTGGCACCCGAGAATTCCACGTTCTTTGTACTGTCCGAAAC',
 'padlock_alt': '/5Phos/TTGCTGCTTTCTGATGCTATCTACGCGGTTAGCACGTANNATACGTCACGGTCCTAGCAAGACTCCACACTCATACCTTGGCACCCGAGAATTCCAGTTCTTTGTACTGTCCGAAACT'}

In [93]:
FGFR1_CDS='''ATGGAGGCAAGGGTCAGTTTGAAAAGGAGGATCGAGCTCACTGTGGAGTATCCATGGAGATGTGGAGCCT
TGTCACCAACCTCTAACTGCAGAACTGGGATGTGGAGCTGGAAGTGCCTCCTCTTCTGGGCTGTGCTGGT
CACAGCCACACTCTGCACCGCTAGGCCGTCCCCGACCTTGCCTGAACAAGCCCAGCCCTGGGGAGCCCCT
GTGGAAGTGGAGTCCTTCCTGGTCCACCCCGGTGACCTGCTGCAGCTTCGCTGTCGGCTGCGGGACGATG
TGCAGAGCATCAACTGGCTGCGGGACGGGGTGCAGCTGGCGGAAAGCAACCGCACCCGCATCACAGGGGA
GGAGGTGGAGGTGCAGGACTCCGTGCCCGCAGACTCCGGCCTCTATGCTTGCGTAACCAGCAGCCCCTCG
GGCAGTGACACCACCTACTTCTCCGTCAATGTTTCAGATGCTCTCCCCTCCTCGGAGGATGATGATGATG
ATGATGACTCCTCTTCAGAGGAGAAAGAAACAGATAACACCAAACCAAACCCCGTAGCTCCATATTGGAC
ATCCCCAGAAAAGATGGAAAAGAAATTGCATGCAGTGCCGGCTGCCAAGACAGTGAAGTTCAAATGCCCT
TCCAGTGGGACCCCAAACCCCACACTGCGCTGGTTGAAAAATGGCAAAGAATTCAAACCTGACCACAGAA
TTGGAGGCTACAAGGTCCGTTATGCCACCTGGAGCATCATAATGGACTCTGTGGTGCCCTCTGACAAGGG
CAACTACACCTGCATTGTGGAGAATGAGTACGGCAGCATCAACCACACATACCAGCTGGATGTCGTGGAG
CGGTCCCCTCACCGGCCCATCCTGCAAGCAGGGTTGCCCGCCAACAAAACAGTGGCCCTGGGTAGCAACG
TGGAGTTCATGTGTAAGGTGTACAGTGACCCGCAGCCGCACATCCAGTGGCTAAAGCACATCGAGGTGAA
TGGGAGCAAGATTGGCCCAGACAACCTGCCTTATGTCCAGATCTTGAAGACTGCTGGAGTTAATACCACC
GACAAAGAGATGGAGGTGCTTCACTTAAGAAATGTCTCCTTTGAGGACGCAGGGGAGTATACGTGCTTGG
CGGGTAACTCTATCGGACTCTCCCATCACTCTGCATGGTTGACCGTTCTGGAAGCCCTGGAAGAGAGGCC
GGCAGTGATGACCTCGCCCCTGTACCTGGAGATCATCATCTATTGCACAGGGGCCTTCCTCATCTCCTGC
ATGGTGGGGTCGGTCATCGTCTACAAGATGAAGAGTGGTACCAAGAAGAGTGACTTCCACAGCCAGATGG
CTGTGCACAAGCTGGCCAAGAGCATCCCTCTGCGCAGACAGGTAACAGTGTCTGCTGACTCCAGTGCATC
CATGAACTCTGGGGTTCTTCTGGTTCGGCCATCACGGCTCTCCTCCAGTGGGACTCCCATGCTAGCAGGG
GTCTCTGAGTATGAGCTTCCCGAAGACCCTCGCTGGGAGCTGCCTCGGGACAGACTGGTCTTAGGCAAAC
CCCTGGGAGAGGGCTGCTTTGGGCAGGTGGTGTTGGCAGAGGCTATCGGGCTGGACAAGGACAAACCCAA
CCGTGTGACCAAAGTGGCTGTGAAGATGTTGAAGTCGGACGCAACAGAGAAAGACTTGTCAGACCTGATC
TCAGAAATGGAGATGATGAAGATGATCGGGAAGCATAAGAATATCATCAACCTGCTGGGGGCCTGCACGC
AGGATGGTCCCTTGTATGTCATCGTGGAGTATGCCTCCAAGGGCAACCTGCGGGAGTACCTGCAGGCCCG
GAGGCCCCCAGGGCTGGAATACTGCTACAACCCCAGCCACAACCCAGAGGAGCAGCTCTCCTCCAAGGAC
CTGGTGTCCTGCGCCTACCAGGTGGCCCGAGGCATGGAGTATCTGGCCTCCAAGAAGTGCATACACCGAG
ACCTGGCAGCCAGGAATGTCCTGGTGACAGAGGACAATGTGATGAAGATAGCAGACTTTGGCCTCGCACG
GGACATTCACCACATCGACTACTATAAAAAGACAACCAACGGCCGACTGCCTGTGAAGTGGATGGCACCC
GAGGCATTATTTGACCGGATCTACACCCACCAGAGTGATGTGTGGTCTTTCGGGGTGCTCCTGTGGGAGA
TCTTCACTCTGGGCGGCTCCCCATACCCCGGTGTGCCTGTGGAGGAACTTTTCAAGCTGCTGAAGGAGGG
TCACCGCATGGACAAGCCCAGTAACTGCACCAACGAGCTGTACATGATGATGCGGGACTGCTGGCATGCA
GTGCCCTCACAGAGACCCACCTTCAAGCAGCTGGTGGAAGACCTGGACCGCATCGTGGCCTTGACCTCCA
ACCAGGAGTACCTGGACCTGTCCATGCCCCTGGACCAGTACTCCCCCAGCTTTCCCGACACCCGGAGCTC
TACGTGCTCCTCAGGGGAGGATTCCGTCTTCTCTCATGAGCCGCTGCCCGAGGAGCCCTGCCTGCCCCGA
CACCCAGCCCAGCTTGCCAATGGCGGACTCAAACGCCGCTGA'''.replace('\n','')

In [251]:
FGFR1_flex=flex_padlock(FGFR1_CDS, "c.719G>A")
FGFR1_flex

118 22 22


{'LHS_ref': 'GACTCCACACTCATACCTTGGCACCCGAGAATTCCAATGATGCTCCAGGTGGCATAAC',
 'target_LHS_ref': 'ATGATGCTCCAGGTGGCATAAC',
 'LHS_mut': 'GACTCCACACTCATACCTTGGCACCCGAGAATTCCATGATGCTCCAGGTGGCATAACT',
 'target_LHS_mut': 'TGATGCTCCAGGTGGCATAACT',
 'target_RHS': 'GGACCTTGTAGCCTCCAATTCT',
 'RHS': '/5Phos/GGACCTTGTAGCCTCCAATTCTACGCGGTTAGCACGTANNATACGTCACGGTCCTAGCAA',
 'padlock_ref': '/5Phos/GGACCTTGTAGCCTCCAATTCTACGCGGTTAGCACGTANNATACGTCACGGTCCTAGCAAGACTCCACACTCATACCTTGGCACCCGAGAATTCCAATGATGCTCCAGGTGGCATAAC',
 'padlock_alt': '/5Phos/GGACCTTGTAGCCTCCAATTCTACGCGGTTAGCACGTANNATACGTCACGGTCCTAGCAAGACTCCACACTCATACCTTGGCACCCGAGAATTCCATGATGCTCCAGGTGGCATAACT'}

In [ ]:
### https://cancer.sanger.ac.uk/cosmic/mutation/overview?id=130818805

In [200]:
PTPRC='''ATGACCATGTATTTGTGGCTTAAACTCTTGGCATTTGGCTTTGCCTTTCTGGACACAGAAGTATTTGTGA
CAGGGCAAAGCCCAACACCTTCCCCCACTGGATTGACTACAGCAAAGATGCCCAGTGTTCCACTTTCAAG
TGACCCCTTACCTACTCACACCACTGCATTCTCACCCGCAAGCACCTTTGAAAGAGAAAATGACTTCTCA
GAGACCACAACTTCTCTTAGTCCAGACAATACTTCCACCCAAGTATCCCCGGACTCTTTGGATAATGCTA
GTGCTTTTAATACCACAGGTGTTTCATCAGTACAGACGCCTCACCTTCCCACGCACGCAGACTCGCAGAC
GCCCTCTGCTGGAACTGACACGCAGACATTCAGCGGCTCCGCCGCCAATGCAAAACTCAACCCTACCCCA
GGCAGCAATGCTATCTCAGATGTCCCAGGAGAGAGGAGTACAGCCAGCACCTTTCCTACAGACCCAGTTT
CCCCATTGACAACCACCCTCAGCCTTGCACACCACAGCTCTGCTGCCTTACCTGCACGCACCTCCAACAC
CACCATCACAGCGAACACCTCAGATGCCTACCTTAATGCCTCTGAAACAACCACTCTGAGCCCTTCTGGA
AGCGCTGTCATTTCAACCACAACAATAGCTACTACTCCATCTAAGCCAACATGTGATGAAAAATATGCAA
ACATCACTGTGGATTACTTATATAACAAGGAAACTAAATTATTTACAGCAAAGCTAAATGTTAATGAGAA
TGTGGAATGTGGAAACAATACTTGCACAAACAATGAGGTGCATAACCTTACAGAATGTAAAAATGCGTCT
GTTTCCATATCTCATAATTCATGTACTGCTCCTGATAAGACATTAATATTAGATGTGCCACCAGGGGTTG
AAAAGTTTCAGTTACATGATTGTACACAAGTTGAAAAAGCAGATACTACTATTTGTTTAAAATGGAAAAA
TATTGAAACCTTTACTTGTGATACACAGAATATTACCTACAGATTTCAGTGTGGTAATATGATATTTGAT
AATAAAGAAATTAAATTAGAAAACCTTGAACCCGAACATGAGTATAAGTGTGACTCAGAAATACTCTATA
ATAACCACAAGTTTACTAACGCAAGTAAAATTATTAAAACAGATTTTGGGAGTCCAGGAGAGCCTCAGAT
TATTTTTTGTAGAAGTGAAGCTGCACATCAAGGAGTAATTACCTGGAATCCCCCTCAAAGATCATTTCAT
AATTTTACCCTCTGTTATATAAAAGAGACAGAAAAAGATTGCCTCAATCTGGATAAAAACCTGATCAAAT
ATGATTTGCAAAATTTAAAACCTTATACGAAATATGTTTTATCATTACATGCCTACATCATTGCAAAAGT
GCAACGTAATGGAAGTGCTGCAATGTGTCATTTCACAACTAAAAGTGCTCCTCCAAGCCAGGTCTGGAAC
ATGACTGTCTCCATGACATCAGATAATAGTATGCATGTCAAGTGTAGGCCTCCCAGGGACCGTAATGGCC
CCCATGAACGTTACCATTTGGAAGTTGAAGCTGGAAATACTCTGGTTAGAAATGAGTCGCATAAGAATTG
CGATTTCCGTGTAAAAGATCTTCAATATTCAACAGACTACACTTTTAAGGCCTATTTTCACAATGGAGAC
TATCCTGGAGAACCCTTTATTTTACATCATTCAACATCTTATAATTCTAAGGCACTGATAGCATTTCTGG
CATTTCTGATTATTGTGACATCAATAGCCCTGCTTGTTGTTCTCTACAAAATCTATGATCTACATAAGAA
AAGATCCTGCAATTTAGATGAACAGCAGGAGCTTGTTGAAAGGGATGATGAAAAACAACTGATGAATGTG
GAGCCAATCCATGCAGATATTTTGTTGGAAACTTATAAGAGGAAGATTGCTGATGAAGGAAGACTTTTTC
TGGCTGAATTTCAGAGCATCCCGCGGGTGTTCAGCAAGTTTCCTATAAAGGAAGCTCGAAAGCCCTTTAA
CCAGAATAAAAACCGTTATGTTGACATTCTTCCTTATGATTATAACCGTGTTGAACTCTCTGAGATAAAC
GGAGATGCAGGGTCAAACTACATAAATGCCAGCTATATTGATGGTTTCAAAGAACCCAGGAAATACATTG
CTGCACAAGGTCCCAGGGATGAAACTGTTGATGATTTCTGGAGGATGATTTGGGAACAGAAAGCCACAGT
TATTGTCATGGTCACTCGATGTGAAGAAGGAAACAGGAACAAGTGTGCAGAATACTGGCCGTCAATGGAA
GAGGGCACTCGGGCTTTTGGAGATGTTGTTGTAAAGATCAACCAGCACAAAAGATGTCCAGATTACATCA
TTCAGAAATTGAACATTGTAAATAAAAAAGAAAAAGCAACTGGAAGAGAGGTGACTCACATTCAGTTCAC
CAGCTGGCCAGACCACGGGGTGCCTGAGGATCCTCACTTGCTCCTCAAACTGAGAAGGAGAGTGAATGCC
TTCAGCAATTTCTTCAGTGGTCCCATTGTGGTGCACTGCAGTGCTGGTGTTGGGCGCACAGGAACCTATA
TCGGAATTGATGCCATGCTAGAAGGCCTGGAAGCCGAGAACAAAGTGGATGTTTATGGTTATGTTGTCAA
GCTAAGGCGACAGAGATGCCTGATGGTTCAAGTAGAGGCCCAGTACATCTTGATCCATCAGGCTTTGGTG
GAATACAATCAGTTTGGAGAAACAGAAGTGAATTTGTCTGAATTACATCCATATCTACATAACATGAAGA
AAAGGGATCCACCCAGTGAGCCGTCTCCACTAGAGGCTGAATTCCAGAGACTTCCTTCATATAGGAGCTG
GAGGACACAGCACATTGGAAATCAAGAAGAAAATAAAAGTAAAAACAGGAATTCTAATGTCATCCCATAT
GACTATAACAGAGTGCCACTTAAACATGAGCTGGAAATGAGTAAAGAGAGTGAGCATGATTCAGATGAAT
CCTCTGATGATGACAGTGATTCAGAGGAACCAAGCAAATACATCAATGCATCTTTTATAATGAGCTACTG
GAAACCTGAAGTGATGATTGCTGCTCAGGGACCACTGAAGGAGACCATTGGTGACTTTTGGCAGATGATC
TTCCAAAGAAAAGTCAAAGTTATTGTTATGCTGACAGAACTGAAACATGGAGACCAGGAAATCTGTGCTC
AGTACTGGGGAGAAGGAAAGCAAACATATGGAGATATTGAAGTTGACCTGAAAGACACAGACAAATCTTC
AACTTATACCCTTCGTGTCTTTGAACTGAGACATTCCAAGAGGAAAGACTCTCGAACTGTGTACCAGTAC
CAATATACAAACTGGAGTGTGGAGCAGCTTCCTGCAGAACCCAAGGAATTAATCTCTATGATTCAGGTCG
TCAAACAAAAACTTCCCCAGAAGAATTCCTCTGAAGGGAACAAGCATCACAAGAGTACACCTCTACTCAT
TCACTGCAGGGATGGATCTCAGCAAACGGGAATATTTTGTGCTTTGTTAAATCTCTTAGAAAGTGCGGAA
ACAGAAGAGGTAGTGGATATTTTTCAAGTGGTAAAAGCTCTACGCAAAGCTAGGCCAGGCATGGTTTCCA
CATTCGAGCAATATCAATTCCTATATGACGTCATTGCCAGCACCTACCCTGCTCAGAATGGACAAGTAAA
GAAAAACAACCATCAAGAAGATAAAATTGAATTTGATAATGAAGTGGACAAAGTAAAGCAGGATGCTAAT
TGTGTTAATCCACTTGGTGCCCCAGAAAAGCTCCCTGAAGCAAAGGAACAGGCTGAAGGTTCTGAACCCA
CGAGTGGCACTGAGGGGCCAGAACATTCTGTCAATGGTCCTGCAAGTCCAGCTTTAAATCAAGGTTCATA
G'''.replace('\n','')

COSMIC: p.M769I; "c.2307G>A"
Bing: M769K, T>A, "c.2306T>A"

In [252]:
PTPRC_flex=flex_padlock(PTPRC, "c.2306T>A")
PTPRC_flex

118 22 22


{'LHS_ref': 'GACTCCACACTCATACCTTGGCACCCGAGAATTCCAAAAGCCCGAGTGCCCTCTTCCA',
 'target_LHS_ref': 'AAAGCCCGAGTGCCCTCTTCCA',
 'LHS_mut': 'GACTCCACACTCATACCTTGGCACCCGAGAATTCCAAAGCCCGAGTGCCCTCTTCCAT',
 'target_LHS_mut': 'AAGCCCGAGTGCCCTCTTCCAT',
 'target_RHS': 'TTGACGGCCAGTATTCTGCACA',
 'RHS': '/5Phos/TTGACGGCCAGTATTCTGCACAACGCGGTTAGCACGTANNATACGTCACGGTCCTAGCAA',
 'padlock_ref': '/5Phos/TTGACGGCCAGTATTCTGCACAACGCGGTTAGCACGTANNATACGTCACGGTCCTAGCAAGACTCCACACTCATACCTTGGCACCCGAGAATTCCAAAAGCCCGAGTGCCCTCTTCCA',
 'padlock_alt': '/5Phos/TTGACGGCCAGTATTCTGCACAACGCGGTTAGCACGTANNATACGTCACGGTCCTAGCAAGACTCCACACTCATACCTTGGCACCCGAGAATTCCAAAGCCCGAGTGCCCTCTTCCAT'}

# order probes in 200uM DDW, hplc if possible

# entrez

https://www.ncbi.nlm.nih.gov/books/NBK25499/#chapter4.EFetch

# get human cds

In [178]:
def get_cds(gene,organism='human'):
    records=[]
    #query ids
    handle = Entrez.esearch(db="nuccore", term="{}[Orgn] AND {}[Gene]".format(organism,gene))
    record = Entrez.read(handle)
    for idx in record["IdList"]:
        try:
            Entrez.email = "A.N.Other@example.com"     # Always tell NCBI who you are
            handle = Entrez.efetch(db="nuccore", id=idx, rettype="fasta_cds_na", retmode="text")
            filename = gene+str(idx)+".fasta"
            if not os.path.isfile(filename):
                with open(filename, "w") as out_handle:
                    out_handle.write(handle.read())
                print(gene,idx," Saved")

                print("Parsing...")
                record = SeqIO.read(filename, "fasta")
                records.append(record)
        except:
            continue
    return records

In [179]:
DNAJB1=get_cds("DNAJB1")

DNAJB1 1890333839  Saved
Parsing...
DNAJB1 1890327943  Saved
Parsing...
DNAJB1 1519245283  Saved
Parsing...
DNAJB1 2194972797  Saved
Parsing...
DNAJB1 568815579  Saved
Parsing...
DNAJB1 2217320975  Saved
Parsing...
DNAJB1 2217320973  Saved
Parsing...
DNAJB1 2217320972  Saved
Parsing...
DNAJB1 2217320971  Saved
Parsing...
DNAJB1 329299062  Saved
Parsing...
DNAJB1 74273663  Saved
Parsing...
DNAJB1 71515761  Saved
Parsing...
DNAJB1 38197192  Saved
Parsing...
DNAJB1 18044281  Saved
Parsing...
DNAJB1 39760203  Saved
Parsing...
DNAJB1 49456314  Saved
Parsing...
DNAJB1 49168457  Saved
Parsing...
DNAJB1 208966143  Saved
Parsing...
DNAJB1 168150060  Saved
Parsing...
DNAJB1 168150059  Saved
Parsing...


In [183]:
for r in DNAJB1:
    if 'CCDS12312.1' in r.description:
        DNAJB1_CDS=r.seq

In [187]:
186*3-1

557

In [194]:
DNAJB1_CDS[555:186*3]

Seq('TCC')

In [264]:
# control probes
# DNAJB1, S186S, 
DNAJB1_flex=flex_padlock(DNAJB1_CDS, "c.558C>A")

118 22 22


In [ ]:
# write to file

In [ ]:
KRAS_flex=flex_padlock(KRAS_CDS,'c.34G>T')
FGFR1_flex=flex_padlock(FGFR1_CDS, "c.719G>A")
TP63_flex=flex_padlock(TP63_CDS,'c.1090G>A')
PTPRC_flex=flex_padlock(PTPRC, "c.2306T>A")
DNAJB1_flex=flex_padlock(DNAJB1_CDS, "c.558C>A")

In [289]:
targets={'KRAS':{'CDS':KRAS_CDS,'mutation_list':['c.34G>T',]},
         'FGFR1':{'CDS':FGFR1_CDS, 'mutation_list':["c.719G>A",]},
         'TP63':{'CDS':TP63_CDS,'mutation_list':['c.1090G>A',]},
         "PTPRC":{'CDS':PTPRC, 'mutation_list':["c.2306T>A",]},
         "DNAJB1":{'CDS':DNAJB1_CDS, 'mutation_list':["c.558C>A",]},
          }

In [290]:
output=dict()
for gene, info in targets.items():
    CDS=info['CDS']
    mutation_list=info['mutation_list']
    for mut in mutation_list:
        output[gene+"_"+mut]=flex_padlock(CDS, mut)
df=pd.DataFrame.from_dict(output,orient='index')


In [291]:
df.to_csv('flex_dna_probes_230300.csv')

In [298]:
to_order={}
for k, i in output.items():
    for kk,ii in i.items():
        to_order[kk+'_'+k]=ii
ddf=pd.DataFrame.from_dict(to_order,orient="index")


In [300]:
ddf['len']=ddf[0].apply(len)

In [304]:
ddf.sort_index(inplace=True)

In [306]:
ddf.to_csv('flex_dan_probes_230310_toorder.csv')